In [1]:
from IPython.display import clear_output

In [2]:
# %pip install torch torchvision pillow spacy numpy
# %pip install torchtext
# %pip install pycocotools

In [3]:
import os
import math
import random

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.data import DataLoader, Dataset, Subset
from torchvision.datasets import MNIST

from tqdm import tqdm

from PIL import Image
import spacy

## Building the tokenizer and vocabulary

In [4]:
spacy_eng = spacy.load("en_core_web_sm")

In [5]:
def word_tokenize(text):
    return [tok.text.lower() for tok in spacy_eng.tokenizer(text)]

In [34]:
numerical_to_word_numbers = {
    0: "Zero",
    1: "One",
    2: "Two",
    3: "Three",
    4: "Four",
    5: "Five",
    6: "Six",
    7: "Seven",
    8: "Eight",
    9: "Nine"
}

possible_caption_templates = [
    "This is an image of the number <number>",
    "We can see the number <number> in this image",
    "Number <number> is displayed here",
    "In this image, the number <number> is shown",
    "Here, you can see the number <number>",
    "The image features the number <number>",
    "Displayed in the image is the number <number>"
]

In [35]:
# Define the vocabulary and tokenizer
word_to_index = {'<PAD>':0, '<SOS>': 1, '<EOS>': 2, '<UNK>': 3}

for text in tqdm(possible_caption_templates+list(numerical_to_word_numbers.values())):
    tokens = word_tokenize(text.lower())
    for token in tokens:
        if token not in word_to_index:
            idx = len(word_to_index)
            word_to_index[token] = idx

index_to_word = {it: k for k, it in word_to_index.items()}

100%|██████████| 17/17 [00:00<00:00, 257412.16it/s]


In [36]:
word_tokenize('<SOS> Testing tokenization breakdown <EOS>')  # We will manually add tokens for <EOS> and <SOS> etc after tokenization to avoid them breaking up.

['<', 'sos', '>', 'testing', 'tokenization', 'breakdown', '<', 'eos', '>']

## Defining the dataset

In [37]:
def get_random_caption(num):

    template_idx = random.randint(0, len(possible_caption_templates)-1)
    template = possible_caption_templates[template_idx]

    caption = template.replace('<number>', numerical_to_word_numbers[num])

    return caption


def convert_sentence_to_idxs(sentence):

    words = word_tokenize(sentence)
    idxs = [word_to_index[word] for word in words]

    return idxs


def convert_idxs_to_sentence(idxs):

    words = [index_to_word[idx] for idx in idxs]
    return ' '.join(words)


class MNISTCaptionsDataset(Dataset):

    def __init__(self, *args, **kwargs):

        self.mnist = MNIST(*args, **kwargs)
        self.max_seq_len = 50

    def __len__(self):
        return len(self.mnist)

    def __getitem__(self, idx):

        img, label = self.mnist[idx]
        caption = get_random_caption(label)

        caption_idxs = convert_sentence_to_idxs(caption.lower().strip())
        if len(caption_idxs) > self.max_seq_len - 2:  # 2 for SOS and EOS
            caption_idxs = caption_idxs[:self.max_seq_len-2]

        padding_len = self.max_seq_len - len(caption_idxs) - 2  # need to pad to make it to seq_len. All captions Need to be of same length so they can be stacked

        caption_numeric = (
            [word_to_index['<SOS>']]+
            caption_idxs+
            [word_to_index['<EOS>']]+
            [word_to_index['<PAD>']]*padding_len
        )

        caption_numeric = torch.Tensor(caption_numeric)

        return img, caption_numeric, caption



## Defining the Model

In [38]:
class EncoderCNN(nn.Module):
    def __init__(self, embed_size):
        super(EncoderCNN, self).__init__()

        # Normally, it would be a good idea to use a pre-trained model as a backbone for feature extraction
        # However, most pretrained models expect RGB images and ones that are bigger in size than 28x28
        # For this reason, you have been provided with commented code to see how we could have used
        # resnet backbone in case of some other dataset but we will use our own custom CNN
        # model for feature extraction.
        # We could have converted MNIST images to RGB and upscaled their size but that would have
        # drastically increased the training time.

        # self.resnet = models.resnet50(pretrained=True).requires_grad_(False)  # resnet embedding backbone
        # self.resnet.fc = nn.Linear(self.resnet.fc.in_features, embed_size)
        # self.relu = nn.ReLU()
        # self.times = []

        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, embed_size)
        self.relu = nn.ReLU()

    def forward(self, x):

        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = x.view(-1, 64 * 7 * 7)
        x = self.relu(self.fc1(x))
        embedding = self.fc2(x)
        embedding = self.relu(embedding)

        return embedding

    # def forward(self, images):

    #     features = self.resnet(images)
    #     return self.relu(features)


class DecoderRNN(nn.Module):
    def __init__(self, embed_size, hidden_size, vocab_size, num_layers):
        super(DecoderRNN, self).__init__()
        self.embed = nn.Embedding(vocab_size, embed_size)

        self.rnn = nn.LSTM(embed_size, hidden_size, num_layers, batch_first=True)

        # NOTE: We can replace the LSTM architecture by RNN or GRU by replacing the above line by one of the following

        # self.rnn = nn.RNN(embed_size, hidden_size, num_layers, batch_first=True)
        # self.rnn = nn.GRU(embed_size, hidden_size, num_layers, batch_first=True)

        self.linear = nn.Sequential(
            nn.Linear(hidden_size, 1024),
            nn.Linear(1024, vocab_size)
        )

    def forward(self, features, captions):

        embeddings = self.embed(captions)
        embeddings = torch.cat((features.unsqueeze(1), embeddings), dim=-2)
        hiddens, _ = self.rnn(embeddings)
        outputs = self.linear(hiddens)
        return outputs


class ImageCaptioner(nn.Module):

    def __init__(self, embed_size, hidden_size, vocab_size, num_layers):
        super(ImageCaptioner, self).__init__()
        self.encoder = EncoderCNN(embed_size)
        self.decoder = DecoderRNN(embed_size, hidden_size, vocab_size, num_layers)

    def forward(self, images, captions):
        features = self.encoder(images)
        outputs = self.decoder(features, captions)
        return outputs

    def caption_image(self, image, max_length=50):

        result_caption = []

        with torch.no_grad():
            x = self.encoder(image)
            states = None

            for _ in range(max_length):

                hiddens, states = self.decoder.rnn(x, states)
                output = self.decoder.linear(hiddens.squeeze(0))

                predicted = output.argmax(0)
                pred_token = index_to_word[predicted.item()]
                result_caption.append(pred_token)
                x = self.decoder.embed(predicted).unsqueeze(0)

                if pred_token == "<EOS>":
                    break

        return result_caption

## Initializing the model and dataset

In [39]:
train_transforms = transforms.Compose(
        [
            # If we want to upscale MNIST images and convert to RGB
            # transforms.Grayscale(num_output_channels=3),
            # transforms.Resize((299, 299)),

            transforms.ToTensor(),
        ]
    )

batch_size = 128
total_dataset = MNISTCaptionsDataset(root='./datasets', train=False, download=True, transform=train_transforms)

train_dataset = Subset(total_dataset, range(len(total_dataset) - 100))
test_dataset = Subset(total_dataset, range(len(total_dataset) - 100, len(total_dataset)))

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

clear_output()

In [40]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [52]:
embed_size = 128
hidden_size = 128
vocab_size = len(word_to_index)
num_decoder_layers = 1
learning_rate = 5e-5
num_epochs = 150

In [53]:
# initialize model, loss etc
model = ImageCaptioner(embed_size, hidden_size, vocab_size, num_decoder_layers).to(device)
criterion = nn.CrossEntropyLoss(ignore_index=word_to_index['<PAD>'])  # ignore pad token loss calculations
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

## Pre-training Testing

In [54]:
test_samples = [test_dataset[i] for i in range(10, 13)]
test_imgs, test_caption_tensors, test_captions = list(zip(*test_samples))

test_imgs = torch.stack(test_imgs, 0).to(device)

In [55]:
transforms.functional.to_pil_image(test_imgs.reshape([1, 1, -1, 28])[0])

In [83]:
test_captions

('We can see the number Eight in this image',
 'Here, you can see the number Zero',
 'The image features the number One')

In [57]:
# Predicted captions

model.eval()
captions = []

for img in test_imgs:
    with torch.no_grad():
        caption = model.caption_image(img.unsqueeze(0))
        caption = ' '.join(caption)
        captions.append(caption)

print(('\n'+'-'*20+'\n').join(captions))

is , two < < , < , < < , < , < < , < , < < , < , < < , < , < < , < , < < , < , < < , < , < < , < , < <
--------------------
is , two < < , < , < < , < , < < , < , < < , < , < < , < , < < , < , < < , < , < < , < , < < , < , < <
--------------------
is , two < < , < , < < , < , < < , < , < < , < , < < , < , < < , < , < < , < , < < , < , < < , < , < <


## Training the model

In [58]:
for epoch in range(num_epochs):

    model.train()

    for idx, (imgs, captions, _) in tqdm(
        enumerate(train_loader), total=len(train_loader), leave=False
    ):
        imgs = imgs.to(device)
        captions = captions.to(device).type(torch.long)

        outputs = model(imgs, captions[:, :-1])
        loss = criterion(
            outputs.reshape(-1, outputs.shape[2]), captions.reshape(-1)
        )

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    if epoch % 10 == 0:
        with torch.no_grad():

            model.eval()
            captions = []

            for img in test_imgs:
                with torch.no_grad():
                    caption = model.caption_image(img.unsqueeze(0))
                    caption = ' '.join(caption)
                    captions.append(caption)

            print(('\n'+'-'*20+'\n').join(captions))

    print(f'\nEpoch: {epoch+1}/{num_epochs}', "Training loss: ", loss.item())

<SOS> the number number <EOS>
--------------------
<SOS> the number number <EOS>
--------------------
<SOS> the number number <EOS>

Epoch: 1/150 Training loss:  2.89863657951355



Epoch: 2/150 Training loss:  1.9204957485198975



Epoch: 3/150 Training loss:  1.1567866802215576



Epoch: 4/150 Training loss:  0.7788854837417603



Epoch: 5/150 Training loss:  0.6105071306228638


...


Epoch: 146/150 Training loss:  0.22036497294902802



Epoch: 147/150 Training loss:  0.20381954312324524



Epoch: 148/150 Training loss:  0.21809279918670654



Epoch: 149/150 Training loss:  0.20756754279136658



Epoch: 150/150 Training loss:  0.20486894249916077


## Testing the results

In [81]:
random_test_img_idxs = random.sample(list(range(len(test_dataset))), 10)
random_test_imgs = [test_dataset[i][0] for i in random_test_img_idxs]

transforms.functional.to_pil_image(torch.cat(random_test_imgs, -1))

In [82]:
with torch.no_grad():
    generated_captions = [model.caption_image(t_img.to(device)) for t_img in random_test_imgs]

generated_captions = [' '.join(tokens) for tokens in generated_captions]

generated_captions

['<SOS> in this image , the number four is shown <EOS>',
 '<SOS> number seven is displayed here <EOS>',
 '<SOS> number five is displayed here <EOS>',
 '<SOS> we can see the number eight in this image <EOS>',
 '<SOS> number eight is displayed here <EOS>',
 '<SOS> here , you can see the number zero <EOS>',
 '<SOS> number nine is displayed here <EOS>',
 '<SOS> number zero is displayed here <EOS>',
 '<SOS> here , you can see the number six <EOS>',
 '<SOS> we can see the number eight in this image <EOS>']